# Agentic Sales Pipeline

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

## Load API tokens for our 3rd party APIs

In [2]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'config/lead_qualification_agents.yaml',
    'lead_tasks': 'config/lead_qualification_tasks.yaml',
    'email_agents': 'config/email_engagement_agents.yaml',
    'email_tasks': 'config/email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

In [8]:
for key in configs.keys():
    print(key)
    for k,v in configs[key].items():
        print(k,': ',v)
    print()

lead_agents
lead_data_agent :  {'role': 'Lead Data Specialist\n', 'goal': 'Collect and analyze personal and company-level data for each lead.\n', 'backstory': 'You are a data specialist with a keen eye for detail, responsible for gathering and analyzing both personal and company data to determine the potential of each lead.\n', 'verbose': True, 'allow_delegation': False}
cultural_fit_agent :  {'role': 'Cultural Fit Analyst\n', 'goal': "Assess the cultural alignment between the lead's company and our organization, and evaluate the overall fit.\n", 'backstory': 'You specialize in understanding company cultures and values, ensuring that partnerships are not only strategic but also culturally aligned for long-term success.\n', 'verbose': True, 'allow_delegation': False}
scoring_validation_agent :  {'role': 'Lead Scorer and Validator\n', 'goal': 'Aggregate data, assign a lead score based on predefined criteria, and validate the scoring accuracy.\n', 'backstory': 'You are an expert in evalua

## Create Pydantic Models for Structured Output

In [9]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")

## Importing Tools

In [10]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

## Lead Qualification Crew, Agents and Tasks

In [11]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)

## Email Engagement Crew

In [12]:
# Creating Agents
email_content_specialist = Agent(
  config=email_agents_config['email_content_specialist']
)

engagement_strategist = Agent(
  config=email_agents_config['engagement_strategist']
)

# Creating Tasks
email_drafting = Task(
  config=email_tasks_config['email_drafting'],
  agent=email_content_specialist
)

engagement_optimization = Task(
  config=email_tasks_config['engagement_optimization'],
  agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
  agents=[
    email_content_specialist,
    engagement_strategist
  ],
  tasks=[
    email_drafting,
    engagement_optimization
  ],
  verbose=True
)

2024-11-15 11:10:02,285 - 139751051729792 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Creating Complete Sales Flow

In [24]:
from crewai import Flow
from crewai.flow.flow import listen, start

class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "Ikram Khan",
                    "job_title": "Freenlancer Ai engineer",
                    "company": "Upwork",
                    "email": "ikram@upwork.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score['lead_score'].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()

## Plotting the Flow

In [25]:
flow.plot()

Graph saved as crewai_flow_graph.html


In [26]:
from IPython.display import IFrame

IFrame(src='./crewai_flow.html', width='150%', height=600)

## Flow Kickoff

In [27]:
emails = await flow.kickoff()

2024-11-15 11:15:01,973 - 139751051729792 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Lead Data Specialist
## Task: Collect and analyze the following information about the lead:
- Personal Information:
  - Name: Obtain the full name of the lead.
  - Job Title: Determine the lead's current job title.
  - Role Relevance: Assess how relevant the lead's role is to the decision-making process on a scale from 0 to 10.
  - Professional Background: Optionally, gather a brief description of the lead's professional background.

- Company Information:
  - Company Name: Identify the name of the company the lead works for.
  - Industry: Determine the industry in which the company operates.
  - Company Size: Estimate the size of the company in terms of employee count.
  - Revenue: If available, collect information on the annual revenue of the company.
  - Market Presence: Evaluate the company's market presence on a scale from 0 to 10.

- Our Company and Product:
  - Company Name: CrewAI
  - Product: Multi-Agent Orchestration Platform
  - ICP: Enterprise companies looking int



# Agent: Lead Data Specialist
## Thought: Thought: Since I have not yet gathered enough information regarding Ikram Khan, especially his professional background, I will now focus on searching for details about Upwork, including the company name, industry, company size, and revenue.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Upwork company information industry size revenue\"}"
## Tool Output: 

Search results: Title: Upwork Revenue and Client Stats (2024)
Link: https://backlinko.com/upwork-users
Snippet: Upwork's marketplace generated $586.1 million in revenue in 2023, representing 85.05% of the company's total revenue. Upwork Marketplace Revenue.
---
Title: Upwork Global Company Profile 2024: Stock Performance & Earnings
Link: https://pitchbook.com/profiles/company/181058-86
Snippet: As of 30-Sep-2024, Upwork Global has a trailing 12-month revenue of $762M. EV In Thousands, USD. TTM 1,247,381. FY 2023 1,837,634.
---
Title: Upwork Reports First Quarter 20



# Agent: Lead Data Specialist
## Thought: Thought: I have gathered some useful information about Upwork including the number of employees and the industry it operates in, but I need to confirm those details and find more specific data points to match the criteria needed in the final answer.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://en.wikipedia.org/wiki/Upwork\"}"
## Tool Output: 
Upwork - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
		Navigation
Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us
		Contribute
HelpLearn to editCommunity portalRecent changesUpload file
Search
Search
Appearance
Donate
Create account
Log in
Personal tools
Donate Create account Log in
		Pages for logged out editors learn more
ContributionsTalk
Contents
move to sidebar
hide
(Top)
1
History
2
Service and business model
3
Size, scope, and changes
4
References
5
External links
Toggle the table of contents
Upwork
17 languages
العر



# Agent: Lead Data Specialist
## Final Answer: 
**Data Report: Lead Information**

**Personal Information:**
- **Name:** Ikram Khan
- **Job Title:** AI Engineer (currently listed as Freelancer Ai Engineer on Upwork)
- **Role Relevance:** 8 (Ikram Khan's expertise in AI engineering is highly relevant to decision-making regarding AI automation solutions.)
- **Professional Background:** Ikram Khan has a robust background in AI, with experience in data analytics, statistics, and machine learning.

**Company Information:**
- **Company Name:** Upwork Global Inc.
- **Industry:** Freelance Marketplace (Human Capital Services)
- **Company Size:** Approximately 800 employees.
- **Revenue:** The annual revenue is reported at approximately $421.6 million for 2024.
- **Market Presence:** 9 (Upwork has significant market presence globally, serving both clients and freelancers internationally, facilitating millions of transactions across numerous sectors.) 

**Our Company and Product:**
- **Company



# Agent: Cultural Fit Analyst
## Thought: Thought: I found a link that may provide a comprehensive overview of Upwork's mission and values. I will read the content from this source to gather necessary information.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.upwork.com/about/our-impact\"}"
## Tool Output: 
Just a moment...Enable JavaScript and cookies to continue


# Agent: Cultural Fit Analyst
## Thought: Thought: I am unable to access the content directly from the Upwork website due to restrictions. I should try reading another relevant source that may outline the company's mission, vision, and values.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.upwork.com/careers/life-at-upwork\"}"
## Tool Output: 
Just a moment...Enable JavaScript and cookies to continue


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: Search the internet(**kwargs: An



# Agent: Cultural Fit Analyst
## Final Answer: 
**Cultural Fit Assessment Report**

- **Cultural Fit Score:** 8/10
- **Cultural Values:**
  - Community first, economic opportunity, innovation, and excellence in execution.
- **Strategic Alignment:**
  - Both organizations emphasize the importance of technology in driving productivity and innovation, making them well aligned in terms of strategic objectives.
- **Comments:**
  - Upwork's strong focus on community success and innovation complements CrewAI's overarching goals, although the nature of their business models may differ. The cultural alignment is strong, signaling a promising collaborative potential.


# Agent: Lead Scorer and Validator
## Task: Aggregate the collected data and perform the following steps: - Score Calculation: Based on predefined criteria, calculate a final lead score (0-100). Consider factors such as:
  - Role Relevance
  - Company Size
  - Market Presence
  - Cultural Fit
- Scoring Criteria Documentation: Li



# Agent: Lead Scorer and Validator
## Thought: Thought: I have obtained relevant information about the job title "Freelancer AI Engineer" and its relevance in enterprise automation. Now, I need to evaluate cultural fit, focusing on how CrewAI's product aligns with the market presence of Upwork and the potential fit of the lead based on its use case.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Cultural fit for automation platforms in freelancing and enterprises\"}"
## Tool Output: 

Search results: Title: The Future of Tech Freelancers and Companies' Use of Technology ...
Link: https://www.linkedin.com/pulse/future-tech-freelancers-companies-use-technology-find-blauenfeldt-3g93f
Snippet: These platforms not only facilitate faster hiring but also contribute to long-term success by predicting cultural fit and potential for growth ...
---
Title: The Importance of a Cultural Fit When Hiring - Business.com
Link: https://www.business.com/articles/hire-for-cultur

2024-11-15 11:16:04,853 - 139751051729792 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Email Content Writer
## Task: Craft a highly personalized email using the lead's name, job title, company information, and any relevant personal or company achievements. The email should speak directly to the lead's interests and the needs of their company. This is not as cold outreach as it is a follow up to a lead form, so keep it short and to the point. Don't use any salutations or closing remarks, nor too complex sentences.
Our Company and Product: - Company Name: CrewAI - Product: Multi-Agent Orchestration Platform - ICP: Enterprise companies looking into Agentic automation. - Pitch: We are a platform that allows you to orchestrate AI Agents for automations to any vertical.
Use the following information: Personal Info: {'name': 'Ikram Khan', 'job_title': 'Freelancer AI Engineer', 'role_relevance': 10, 'professional_background': None} Company Info: {'company_name': 'Upwork', 'industry': 'Freelancing', 'company_size': 1000, 'revenue': None, 'market_presence': 9} Lead Score:

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [28]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0162


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,108107,101896,6211,43


In [29]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0003


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,1735,1382,353,3


## Inspecting Results

In [30]:
scores = flow.state["score_crews_results"]

In [31]:
import pandas as pd
from IPython.display import display, HTML

lead_scoring_result = scores[0].pydantic

# Create a dictionary with the nested structure flattened
data = {
    'Name': lead_scoring_result.personal_info.name,
    'Job Title': lead_scoring_result.personal_info.job_title,
    'Role Relevance': lead_scoring_result.personal_info.role_relevance,
    'Professional Background': lead_scoring_result.personal_info.professional_background,
    'Company Name': lead_scoring_result.company_info.company_name,
    'Industry': lead_scoring_result.company_info.industry,
    'Company Size': lead_scoring_result.company_info.company_size,
    'Revenue': lead_scoring_result.company_info.revenue,
    'Market Presence': lead_scoring_result.company_info.market_presence,
    'Lead Score': lead_scoring_result.lead_score.score,
    'Scoring Criteria': ', '.join(lead_scoring_result.lead_score.scoring_criteria),
    'Validation Notes': lead_scoring_result.lead_score.validation_notes
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Reset the index to turn the original column names into a regular column
df = df.reset_index()

# Rename the index column to 'Attribute'
df = df.rename(columns={'index': 'Attribute'})

# Create HTML table with bold attributes and left-aligned values
html_table = df.style.set_properties(**{'text-align': 'left'}) \
                     .format({'Attribute': lambda x: f'<b>{x}</b>'}) \
                     .hide(axis='index') \
                     .to_html()

# Display the styled HTML table
display(HTML(html_table))

Attribute,Value
Name,Ikram Khan
Job Title,Freelancer AI Engineer
Role Relevance,10
Professional Background,None
Company Name,Upwork
Industry,Freelancing
Company Size,1000
Revenue,None
Market Presence,9
Lead Score,95


## Results

In [32]:
import textwrap

result_text = emails[0].raw
wrapped_text = textwrap.fill(result_text, width=80)
print(wrapped_text)

Explore the power of seamless AI automation with CrewAI's Multi-Agent
Orchestration Platform. Unlock custom workflows tailored specifically for
freelancing and enhance your productivity. Ready to elevate your project
outcomes?    **Schedule a meeting now** [Insert Meeting Link] to discuss how we
can align CrewAI's capabilities with your goals at Upwork.    Don’t miss out on
optimizing your operations—**Start your journey today** [Insert Sign-Up Link].
The future of automation is here, let’s make it work for you!


## How Complex Can it Get?

In [33]:
from crewai import Flow
from crewai.flow.flow import listen, start, and_, or_, router

class SalesPipeline(Flow):
    
  @start()
  def fetch_leads(self):
    # Pull our leads from the database
    # This is a mock, in a real-world scenario, this is where you would
    # fetch leads from a database
    leads = [
      {
        "lead_data": {
          "name": "Ikram khan",
          "job_title": "Freelancer Ai Engineer",
          "company": "Upwork",
          "email": "ikram@upwork.com",
          "use_case": "Using AI Agent to do better data enrichment."
        },
      },
    ]
    return leads

  @listen(fetch_leads)
  def score_leads(self, leads):
    scores = lead_scoring_crew.kickoff_for_each(leads)
    self.state["score_crews_results"] = scores
    return scores

  @listen(score_leads)
  def store_leads_score(self, scores):
    # Here we would store the scores in the database
    return scores

  @listen(score_leads)
  def filter_leads(self, scores):
    return [score for score in scores if score['lead_score'].score > 70]

  @listen(and_(filter_leads, store_leads_score))
  def log_leads(self, leads):
    print(f"Leads: {leads}")

  @router(filter_leads, paths=["high", "medium", "low"])
  def count_leads(self, scores):
    if len(scores) > 10:
      return 'high'
    elif len(scores) > 5:
      return 'medium'
    else:
      return 'low'

  @listen('high')
  def store_in_salesforce(self, leads):
    return leads

  @listen('medium')
  def send_to_sales_team(self, leads):
    return leads

  @listen('low')
  def write_email(self, leads):
    scored_leads = [lead.to_dict() for lead in leads]
    emails = email_writing_crew.kickoff_for_each(scored_leads)
    return emails

  @listen(write_email)
  def send_email(self, emails):
    # Here we would send the emails to the leads
    return emails

## Plotting the Flow

In [34]:
flow = SalesPipeline()
flow.plot()

Graph saved as crewai_flow_graph.html


In [35]:
from IPython.display import IFrame

IFrame(src='./crewai_flow_complex.html', width='150%', height=600)